# Прогноз символов

<https://proproprogs.ru/neural_network/kak-rekurrentnaya-neyronnaya-set-prognoziruet-simvoly>

In [1]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from keras.layers import SimpleRNN, Input, Dense
from keras.models import Sequential

In [2]:
with open('hw_51.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [3]:
num_characters = 34 #33 буквы + пробел

In [4]:
# создание экземпляра класса Tokenizer (инструмент, который делает 
# «умный» парсинг (разложение на составляющие элементы) указанного текста)
# параметры:
#     num_words – максимальное количество слов (символов), которое вернет Tokenizer (если элементов будет больше, то останутся наиболее повторяющиеся в тексте);
#     filters – исключаемые из текста символы (по умолчанию, следующие: !–»—#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n\r);
#     lower = True – автоматический перевод в нижний регистр для единообразия больших и малых символов;
#     split = '  ' – разделение слов по символу пробела;
#     char_level=False – если False, то текст делится на слова, иначе – на символы.

tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [5]:
# пропустим через него загруженный текст
tokenizer.fit_on_texts(text)

In [6]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'т': 4, 'и': 5, 'а': 6, 'н': 7, 'с': 8, 'в': 9, 'р': 10, 'м': 11, 'л': 12, 'ь': 13, 'д': 14, 'п': 15, 'у': 16, 'ы': 17, 'з': 18, 'я': 19, 'б': 20, 'ч': 21, 'к': 22, 'й': 23, 'ж': 24, 'г': 25, 'ш': 26, 'х': 27, 'ю': 28, 'ц': 29, 'щ': 30, 'э': 31, 'ф': 32, 'ъ': 33}


In [7]:
# преобразуем текст в набор OHE-векторов
inp_chars = 5
data = tokenizer.texts_to_matrix(text)

In [8]:
# вычислим размер обучающего множества
n = data.shape[0]-inp_chars

In [9]:
# сформируем входной тензор и прогнозные значения
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [10]:
# создадим рекуррентную НС с помощью Keras
model = Sequential()
model.add(Input((inp_chars, num_characters))) 
model.add(SimpleRNN(128, activation='tanh')) 
model.add(Dense(num_characters, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               20864     
                                                                 
 dense (Dense)               (None, 34)                4386      
                                                                 
Total params: 25250 (98.63 KB)
Trainable params: 25250 (98.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)


Epoch 1/100


197/197 [==============================] - 2s 3ms/step - loss: 2.9547 - accuracy: 0.1858
Epoch 2/100
197/197 [==============================] - 1s 3ms/step - loss: 2.6182 - accuracy: 0.2685
Epoch 3/100
197/197 [==============================] - 1s 3ms/step - loss: 2.4373 - accuracy: 0.3174
Epoch 4/100
197/197 [==============================] - 1s 3ms/step - loss: 2.3129 - accuracy: 0.3412
Epoch 5/100
197/197 [==============================] - 1s 3ms/step - loss: 2.2219 - accuracy: 0.3612
Epoch 6/100
197/197 [==============================] - 1s 3ms/step - loss: 2.1585 - accuracy: 0.3743
Epoch 7/100
197/197 [==============================] - 1s 3ms/step - loss: 2.1066 - accuracy: 0.3929
Epoch 8/100
197/197 [==============================] - 1s 3ms/step - loss: 2.0664 - accuracy: 0.3910
Epoch 9/100
197/197 [==============================] - 1s 3ms/step - loss: 2.0298 - accuracy: 0.4040
Epoch 10/100
197/197 [==============================] - 1s 3ms/step - loss: 1.9906 - acc

197/197 [==============================] - 1s 3ms/step - loss: 0.6059 - accuracy: 0.8080
Epoch 76/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5983 - accuracy: 0.8102
Epoch 77/100
197/197 [==============================] - 1s 3ms/step - loss: 0.6001 - accuracy: 0.8069
Epoch 78/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5879 - accuracy: 0.8180
Epoch 79/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5798 - accuracy: 0.8072
Epoch 80/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5760 - accuracy: 0.8143
Epoch 81/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5697 - accuracy: 0.8131
Epoch 82/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5641 - accuracy: 0.8188
Epoch 83/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5617 - accuracy: 0.8209
Epoch 84/100
197/197 [==============================] - 1s 3ms/step - loss: 0.5624 - accuracy: 

In [12]:
# объявим вспомогательную функцию, в которой будет выполняться прогноз 
# очередного символа и добавления его в конец начальной строки
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding
 
    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)
 
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении
 
    inp_str += d # дописываем строку
 
  return inp_str

In [16]:
res = buildPhrase("хорош")
print(res)

1/1 [==============================] - 0s 33ms/step
хорошее действием на всю остальность величайшим просто 


# Прогноз cлов

<https://proproprogs.ru/neural_network/delaem-prognoz-slov-rekurrentnoy-setyu-embedding-sloy>

In [17]:
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [18]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [20]:
with open('hw_52.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [21]:
maxWordsCount = 1000

In [22]:
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)

In [23]:
tokenizer.fit_on_texts([texts])

In [24]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('я', 21), ('притягиваю', 1), ('только', 21), ('плохое', 1), ('кому', 2), ('нужен', 2), ('с', 12), ('такой', 5), ('внешностью', 2), ('не', 91)]


In [26]:
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )

In [27]:
inp_words = 3
n = res.shape[0] - inp_words

In [28]:
X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [29]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 256)            256000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_1 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 434280 (1.66 MB)
Trainable params: 434280 (1.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [31]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
52/52 [==============================] - 2s 6ms/step - loss: 6.8537 - accuracy: 0.0397
Epoch 2/50
52/52 [==============================] - 0s 6ms/step - loss: 6.1833 - accuracy: 0.0547
Epoch 3/50
52/52 [==============================] - 0s 6ms/step - loss: 5.7451 - accuracy: 0.0829
Epoch 4/50
52/52 [==============================] - 0s 6ms/step - loss: 5.2495 - accuracy: 0.1310
Epoch 5/50
52/52 [==============================] - 0s 6ms/step - loss: 4.6044 - accuracy: 0.2019
Epoch 6/50
52/52 [==============================] - 0s 6ms/step - loss: 3.9163 - accuracy: 0.3233
Epoch 7/50
52/52 [==============================] - 0s 7ms/step - loss: 3.2582 - accuracy: 0.4826
Epoch 8/50
52/52 [==============================] - 0s 6ms/step - loss: 2.6609 - accuracy: 0.6010
Epoch 9/50
52/52 [==============================] - 0s 6ms/step - loss: 2.1488 - accuracy: 0.6941
Epoch 10/50
52/52 [==============================] - 0s 6ms/step - loss: 1.7192 - accuracy: 0.7662
Epoch 11/50
52/52 [

In [33]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [35]:
res = buildPhrase("всегда смотрите на")
print(res)

1/1 [==============================] - 0s 25ms/step
всегда смотрите на позитив всегда превращай счастье боль печаль и слезы в позитивную энергию либо ты бежишь день либо день бежит за тобой
